<a href="https://colab.research.google.com/github/kaarelkaarelson/AA/blob/master/week6/arabic_w6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 6: Arabic

In [ ]:
# Google Colab setup
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/")

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate
!pip install tensorflow==2.14

from transformers import pipeline, BertTokenizer, AutoConfig, Trainer, TrainingArguments, BertForQuestionAnswering, AutoTokenizer, DataCollatorWithPadding,Trainer, AutoModelForQuestionAnswering
from datasets import load_dataset, Dataset
import pandas as pd
import string
import nltk
import math
import statistics
from huggingface_hub import notebook_login
import pickle
from tqdm import tqdm
import torch
import concurrent.futures
from sklearn.metrics import f1_score
import numpy as np


nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
notebook_login()

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

full_dataset = load_dataset("copenlu/answerable_tydiqa")
full_dataset = full_dataset.rename_column("question_text", "question")
full_dataset = full_dataset.rename_column("document_plaintext", "context")
full_dataset = full_dataset.rename_column("annotations", "answer")
train = full_dataset["train"]
validation = full_dataset["validation"]

# language = 'bengali'
language = 'indonesian'

validation = validation.filter(lambda x: x["language"]==language).select_columns(["question", "context", "answer"])
validation_add_ids = pd.DataFrame(validation).reset_index().rename(columns={"index": "id"})
validation = Dataset.from_pandas(validation_add_ids)

Filter:   0%|          | 0/13325 [00:00<?, ? examples/s]

In [ ]:
# model = pipeline("question-answering", model="kaarelkaarelson/finetuned-bert-base-arabertv2")
# model = pipeline("question-answering", model="kaarelkaarelson/finetuned-AraElectra-Arabic-SQuADv2-QA")
model = pipeline("question-answering", model="kaarelkaarelson/finetuned_bert-base-multilingual-cased")

## Testing

In [ ]:
def predict(validation, model):
  preds = []

  for i in tqdm(range(len(validation))):
    question = validation[i]['question']
    context = validation[i]['context']

    pred = model(question, context)
    preds.append(pred)

  return preds

In [ ]:
def find_best_threshold(preds, true_labels):
    step = 0.0005
    thresholds = [round(step*i, 4) for i in range(2000)]

    best_threshold = 0
    best_f1 = 0

    for threshold in thresholds:
        binary_preds = [1 if pred['score'] > threshold else 0 for pred in preds]
        f1 = f1_score(true_labels, binary_preds)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

In [ ]:
def transform_to_df(preds, scores, threshold):
  true_labels = []
  labels_pred = []

  for i in range(len(validation)):
    if validation[i]["answer"]["answer_start"]==[-1]:
      true_labels.append(False)
    else:
      true_labels.append(True)

    labels_pred.append(1 if preds[i]['score'] > threshold else 0)

  df = pd.DataFrame({"Score":scores, "Class":true_labels, "Prediction":labels_pred})

  return df

In [ ]:
preds = predict(validation, model)

100%|██████████| 1191/1191 [08:39<00:00,  2.29it/s]


In [ ]:
true_labels = [0 if validation[i]["answer"]["answer_start"] == [-1] else 1 for i in range(len(validation))]

scores = [pred['score'] for pred in preds]
threshold = find_best_threshold(preds, true_labels)
df_preds = transform_to_df(preds, scores, threshold)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function calls for accuracy, precision, recall, and F1 score using scikit-learn functions
accuracy = accuracy_score(df_preds['Class'], df_preds['Prediction'])
precision = precision_score(df_preds['Class'], df_preds['Prediction'])
recall = recall_score(df_preds['Class'], df_preds['Prediction'])
f1 = f1_score(df_preds['Class'], df_preds['Prediction'])

# Print the results
print("Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Metrics:
Accuracy: 0.8757346767422334
Precision: 0.8296622613803231
Recall: 0.9463986599664992
F1 Score: 0.8841940532081377


In [ ]:
from evaluate import load

# preds = preds_finetuned_arabert
# preds = preds_finetuned_araelectra
squad_v2_metric = load("squad_v2")
valid = []
res = []
for i in range(len(preds)):
  res.append({})
  res[i]["id"] = str(validation[i]["id"])
  res[i]["no_answer_probability"] = 1 - preds[i]["score"]
  res[i]["no_answer_probability"] = np.float32(res[i]["no_answer_probability"])
  res[i]["prediction_text"] = preds[i]["answer"]
  tmp = {}
  tmp["answers"] = validation[i]["answer"]
  tmp["id"] = str(validation[i]["id"])
  tmp["answers"]["text"] = tmp["answers"]["answer_text"]
  if(tmp["answers"]["answer_start"]==[-1]):
    tmp["answers"]["answer_start"] = []
    tmp["answers"]["text"] = []
  tmp["answers"].pop("answer_text")
  tmp["answers"]["answer_start"] = np.int32(tmp["answers"]["answer_start"])

  valid.append(tmp)

In [ ]:
metrics = squad_v2_metric.compute(predictions=res, references=valid)

In [ ]:
metrics

{'exact': 23.25776658270361,
 'f1': 31.429926262677316,
 'total': 1191,
 'HasAns_exact': 46.39865996649916,
 'HasAns_f1': 62.70191319740148,
 'HasAns_total': 597,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 594,
 'best_exact': 65.15533165407221,
 'best_exact_thresh': 0.8362839818000793,
 'best_f1': 71.64962167280687,
 'best_f1_thresh': 0.9901544451713562}